# Experiment 039: Phase Coherence Bonding

**Question:** Do semantically coherent concepts show phase alignment while conflicting concepts show phase opposition?

**Hypothesis:** AQ combine via superposition; phase determines constructive vs destructive interference.

**Key Predictions:**
- Coherent pairs: coherence > 0.3, interference ratio > 1.1
- Conflicting pairs: coherence < 0.1, interference ratio < 0.9

In [ ]:
!pip install transformers torch scipy matplotlib -q

In [ ]:
import torch
import numpy as np
from scipy.fft import fft
from scipy.stats import ttest_ind, spearmanr
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 1. Load Model

In [ ]:
MODEL_NAME = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
model.eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Loaded {MODEL_NAME}")

## 2. Define Word Pairs

In [ ]:
# Semantically COHERENT pairs (expect constructive interference)
COHERENT_PAIRS = [
    ("hot", "fire"),
    ("cold", "ice"),
    ("happy", "joy"),
    ("sad", "tears"),
    ("fast", "speed"),
    ("big", "giant"),
    ("small", "tiny"),
    ("cat", "meow"),
    ("dog", "bark"),
    ("sun", "bright"),
    ("moon", "night"),
    ("water", "wet"),
    ("doctor", "hospital"),
    ("teacher", "school"),
    ("king", "throne"),
]

# Semantically CONFLICTING pairs (expect destructive interference)
CONFLICTING_PAIRS = [
    ("hot", "cold"),
    ("big", "small"),
    ("fast", "slow"),
    ("happy", "sad"),
    ("true", "false"),
    ("up", "down"),
    ("left", "right"),
    ("good", "bad"),
    ("light", "dark"),
    ("alive", "dead"),
    ("open", "closed"),
    ("full", "empty"),
    ("love", "hate"),
    ("peace", "war"),
    ("rich", "poor"),
]

# NEUTRAL pairs (expect no systematic interference)
NEUTRAL_PAIRS = [
    ("cat", "telephone"),
    ("mountain", "keyboard"),
    ("happy", "purple"),
    ("doctor", "banana"),
    ("ocean", "pencil"),
    ("king", "sandwich"),
    ("tree", "algorithm"),
    ("sun", "carpet"),
    ("music", "brick"),
    ("book", "volcano"),
    ("chair", "dream"),
    ("window", "philosophy"),
    ("garden", "mathematics"),
    ("bridge", "poetry"),
    ("clock", "elephant"),
]

print(f"Coherent pairs: {len(COHERENT_PAIRS)}")
print(f"Conflicting pairs: {len(CONFLICTING_PAIRS)}")
print(f"Neutral pairs: {len(NEUTRAL_PAIRS)}")

## 3. Phase Analysis Functions

In [ ]:
def get_embedding(word: str) -> torch.Tensor:
    """Get embedding for a word."""
    inputs = tokenizer(word, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    # Last layer, average over tokens
    hidden = outputs.hidden_states[-1]
    return hidden.mean(dim=1).squeeze().cpu()

def compute_phase_coherence(emb1: torch.Tensor, emb2: torch.Tensor, n_components: int = 100) -> float:
    """Compute phase coherence via FFT."""
    # FFT decomposition
    fft1 = fft(emb1.numpy())
    fft2 = fft(emb2.numpy())
    
    # Extract phases
    phase1 = np.angle(fft1)
    phase2 = np.angle(fft2)
    
    # Phase difference
    delta_phase = phase1 - phase2
    delta_phase = np.arctan2(np.sin(delta_phase), np.cos(delta_phase))  # Wrap to [-pi, pi]
    
    # Coherence = mean(cos(delta_phase)) for top components
    n = min(n_components, len(delta_phase) // 2)
    coherence = np.cos(delta_phase[:n]).mean()
    
    return float(coherence)

def compute_interference_ratio(emb1: torch.Tensor, emb2: torch.Tensor) -> float:
    """Compute energy interference ratio."""
    E1 = float((emb1 ** 2).sum())
    E2 = float((emb2 ** 2).sum())
    E_combined = float(((emb1 + emb2) ** 2).sum())
    E_expected = E1 + E2
    
    return E_combined / E_expected if E_expected > 0 else 1.0

def analyze_pair(w1: str, w2: str) -> dict:
    """Full analysis of a word pair."""
    emb1 = get_embedding(w1)
    emb2 = get_embedding(w2)
    
    coherence = compute_phase_coherence(emb1, emb2)
    ratio = compute_interference_ratio(emb1, emb2)
    cos_sim = float(torch.nn.functional.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0)))
    
    return {
        'words': (w1, w2),
        'phase_coherence': coherence,
        'interference_ratio': ratio,
        'cosine_similarity': cos_sim
    }

# Test
test = analyze_pair("hot", "fire")
print(f"Test pair 'hot' + 'fire': coherence={test['phase_coherence']:.3f}, ratio={test['interference_ratio']:.3f}")

## 4. Analyze All Pairs

In [ ]:
# Analyze all pairs
print("Analyzing COHERENT pairs:")
coherent_results = []
for w1, w2 in COHERENT_PAIRS:
    result = analyze_pair(w1, w2)
    coherent_results.append(result)
    print(f"  {w1:10} + {w2:10} | coh={result['phase_coherence']:+.3f} | ratio={result['interference_ratio']:.3f}")

print("\nAnalyzing CONFLICTING pairs:")
conflicting_results = []
for w1, w2 in CONFLICTING_PAIRS:
    result = analyze_pair(w1, w2)
    conflicting_results.append(result)
    print(f"  {w1:10} + {w2:10} | coh={result['phase_coherence']:+.3f} | ratio={result['interference_ratio']:.3f}")

print("\nAnalyzing NEUTRAL pairs:")
neutral_results = []
for w1, w2 in NEUTRAL_PAIRS:
    result = analyze_pair(w1, w2)
    neutral_results.append(result)
    print(f"  {w1:10} + {w2:10} | coh={result['phase_coherence']:+.3f} | ratio={result['interference_ratio']:.3f}")

## 5. Statistical Analysis

In [ ]:
# Extract metrics
coh_coherences = [r['phase_coherence'] for r in coherent_results]
con_coherences = [r['phase_coherence'] for r in conflicting_results]
neu_coherences = [r['phase_coherence'] for r in neutral_results]

coh_ratios = [r['interference_ratio'] for r in coherent_results]
con_ratios = [r['interference_ratio'] for r in conflicting_results]
neu_ratios = [r['interference_ratio'] for r in neutral_results]

# Statistics
print("="*60)
print("STATISTICAL SUMMARY")
print("="*60)

print("\n1. PHASE COHERENCE:")
print(f"   Coherent:   mean={np.mean(coh_coherences):+.4f}, std={np.std(coh_coherences):.4f}")
print(f"   Conflicting: mean={np.mean(con_coherences):+.4f}, std={np.std(con_coherences):.4f}")
print(f"   Neutral:    mean={np.mean(neu_coherences):+.4f}, std={np.std(neu_coherences):.4f}")

print("\n2. INTERFERENCE RATIO:")
print(f"   Coherent:   mean={np.mean(coh_ratios):.4f}, std={np.std(coh_ratios):.4f}")
print(f"   Conflicting: mean={np.mean(con_ratios):.4f}, std={np.std(con_ratios):.4f}")
print(f"   Neutral:    mean={np.mean(neu_ratios):.4f}, std={np.std(neu_ratios):.4f}")

# Statistical tests
t_coh, p_coh = ttest_ind(coh_coherences, con_coherences)
t_ratio, p_ratio = ttest_ind(coh_ratios, con_ratios)

print("\n3. T-TESTS (Coherent vs Conflicting):")
print(f"   Phase coherence: t={t_coh:.2f}, p={p_coh:.4f}")
print(f"   Interference ratio: t={t_ratio:.2f}, p={p_ratio:.4f}")

# Correlation
all_coh = coh_coherences + con_coherences + neu_coherences
all_ratio = coh_ratios + con_ratios + neu_ratios
corr, corr_p = spearmanr(all_coh, all_ratio)

print(f"\n4. COHERENCE-RATIO CORRELATION:")
print(f"   Spearman r={corr:.3f}, p={corr_p:.4f}")

## 6. Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Phase Coherence Distribution
ax1 = axes[0]
types = ['Coherent', 'Conflicting', 'Neutral']
means = [np.mean(coh_coherences), np.mean(con_coherences), np.mean(neu_coherences)]
stds = [np.std(coh_coherences), np.std(con_coherences), np.std(neu_coherences)]
colors = ['green', 'red', 'gray']

bars = ax1.bar(types, means, yerr=stds, color=colors, alpha=0.7, capsize=5)
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax1.set_ylabel('Phase Coherence', fontsize=12)
ax1.set_title('Phase Coherence by Pair Type', fontsize=14)
ax1.set_ylim(-0.4, 0.4)

# Plot 2: Interference Ratio Distribution
ax2 = axes[1]
means_r = [np.mean(coh_ratios), np.mean(con_ratios), np.mean(neu_ratios)]
stds_r = [np.std(coh_ratios), np.std(con_ratios), np.std(neu_ratios)]

ax2.bar(types, means_r, yerr=stds_r, color=colors, alpha=0.7, capsize=5)
ax2.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='No interference')
ax2.set_ylabel('Interference Ratio', fontsize=12)
ax2.set_title('Interference Ratio by Pair Type', fontsize=14)
ax2.legend()

# Plot 3: Scatter
ax3 = axes[2]
ax3.scatter(coh_coherences, coh_ratios, c='green', label='Coherent', alpha=0.6, s=80)
ax3.scatter(con_coherences, con_ratios, c='red', label='Conflicting', alpha=0.6, s=80)
ax3.scatter(neu_coherences, neu_ratios, c='gray', label='Neutral', alpha=0.6, s=80)
ax3.axhline(y=1.0, color='black', linestyle='--', alpha=0.3)
ax3.axvline(x=0, color='black', linestyle='--', alpha=0.3)
ax3.set_xlabel('Phase Coherence', fontsize=12)
ax3.set_ylabel('Interference Ratio', fontsize=12)
ax3.set_title('Coherence vs Interference', fontsize=14)
ax3.legend()

plt.tight_layout()
plt.show()

## 7. Summary

In [ ]:
print("="*60)
print("EXPERIMENT 039 SUMMARY")
print("="*60)

# Compute key metrics
coherence_separation = np.mean(coh_coherences) - np.mean(con_coherences)
ratio_separation = np.mean(coh_ratios) - np.mean(con_ratios)

print("\n1. KEY METRICS:")
print(f"   Coherence separation: {coherence_separation:.4f}")
print(f"   Ratio separation: {ratio_separation:.4f}")
print(f"   Coherence t-test p-value: {p_coh:.4f}")
print(f"   Coherence-Ratio correlation: {corr:.3f}")

print("\n2. HYPOTHESIS TEST:")

# Predictions
pred_coh_sep = coherence_separation > 0.1
pred_significant = p_coh < 0.05
pred_correlation = corr > 0.2

print(f"   Coherence separation > 0.1: {pred_coh_sep} ({coherence_separation:.3f})")
print(f"   Statistically significant: {pred_significant} (p={p_coh:.4f})")
print(f"   Positive correlation: {pred_correlation} (r={corr:.3f})")

print("\n3. VERDICT:")
if pred_coh_sep and pred_significant:
    print("   HYPOTHESIS SUPPORTED")
    print("   - Coherent pairs show higher phase coherence")
    print("   - Semantics correlates with phase structure")
    print("   - Phase alignment mechanism validated")
else:
    print("   HYPOTHESIS NOT SUPPORTED")
    if not pred_coh_sep:
        print("   - Coherence separation too small")
    if not pred_significant:
        print("   - Not statistically significant")